# Google Cloud Setup

In [ ]:
!gcloud init

In [ ]:
!gsutil cp ./ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/model.ckpt.* gs://yourbucketname/data/

Set up two environment variables to simplify commands

In [ ]:
!export PROJECT="yourprojectname"
!export YOUR_GCS_BUCKET="yourbucketname"

get the name of your service account with the following command

In [ ]:
!curl -H "Authorization: Bearer $(gcloud auth print-access-token)" https://ml.googleapis.com/v1/projects/${PROJECT}:getConfig

When this command completes, copy the value of tpuServiceAccount (it will look something like your-service-account-12345@cloud-tpu.iam.gserviceaccount.com) and then save it as an environment variable:

In [ ]:
!export TPU_ACCOUNT=your-service-account

Finally, grant the ml.serviceAgent role to your TPU service account:

In [ ]:
!gcloud projects add-iam-policy-binding ${PROJECT} --member serviceAccount:${TPU_ACCOUNT} --role roles/ml.serviceAgent

# Installing TensorFlow Object Detection API

In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 42231 (delta 4), reused 10 (delta 0), pack-reused 42217
Receiving objects: 100% (42231/42231), 549.48 MiB | 33.77 MiB/s, done.
Resolving deltas: 100% (28714/28714), done.


Compile and setup

In [ ]:
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf1/setup.py .
!python -m pip install --use-feature=2020-resolver .

Install dependent packages

In [ ]:
!pip install tf-nightly
!pip install tf-models-official
!pip install tf-models-nightly

Test installation by running the following</br>
If installation is successful, you should see the following output:</br>


> Ran 18 tests in 0.079s</br>
> OK 


In [ ]:
!python object_detection/builders/model_builder_test.py

# Setting up the dataset and Uploading dataset to GCS(Just example)

Below is a quick example for dataset configuration. However, the following training, evaluation and model format transform process is based on our own dataset, which is [Color-Fashion Dataset](https://sites.google.com/site/fashionparsing/dataset). And we have done some necessary processing such as generating tfrecord for training.

In [ ]:
# %cd /content/models/research/
!pwd
!mkdir -p ./tmp/pet_faces_tfrecord
%cd ./tmp/pet_faces_tfrecord
!curl "http://download.tensorflow.org/models/object_detection/pet_faces_tfrecord.tar.gz" | tar xzf -

/content/models/research
/content/models/research/tmp/pet_faces_tfrecord
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  357M  100  357M    0     0  75.9M      0  0:00:04  0:00:04 --:--:-- 75.9M


In [ ]:
!gsutil -m cp -r ./pet_faces* gs://final_my/data/

In [ ]:
!gsutil cp ./models/research/object_detection/data/pet_label_map.pbtxt gs://final_my/data/pet_label_map.pbtxt


Copying file://./models/research/object_detection/data/pet_label_map.pbtxt [Content-Type=application/octet-stream]...
/ [1 files][  1.5 KiB/  1.5 KiB]                                                
Operation completed over 1 objects/1.5 KiB.                                      


# Downloading the SSD MobileNet checkpoint for transfer learning

In [ ]:
%cd /content/models/research/tmp
!curl -O http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03.tar.gz
!tar xzf ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03.tar.gz

!gsutil cp ./model.ckpt.* gs://yourbucketname/data/

/content/models/research/tmp
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 44.4M  100 44.4M    0     0  50.0M      0 --:--:-- --:--:-- --:--:-- 49.9M


# Training a quantized model with Cloud TPUs on Cloud ML Engine

Aftering customed modification to the configuration file considering your own dataset, copy this quantized config file into your GCS bucket

In [ ]:
!gsutil cp /content/models/research/object_detection/samples/configs/ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.config gs://yourbucketname/fashion/pipeline.config

Copying file:///content/models/research/object_detection/samples/configs/ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.config [Content-Type=application/octet-stream]...
/ [1 files][  4.5 KiB/  4.5 KiB]                                                
Operation completed over 1 objects/4.5 KiB.                                      


Before we kick off our training job on Cloud ML Engine, we need to package the Object Detection API, pycocotools, and TF Slim.

In [ ]:
# %cd ..
# !mkdir -p ./tmp/pycocotools
# !rm -r ./tmp/pycocotools
!bash object_detection/dataset_tools/create_pycocotools_package.sh /tmp/pycocotools
!python setup.py sdist
%cd slim
!python setup.py sdist

Login with your own account

In [ ]:
!gcloud auth login

In [ ]:
!gcloud config set project yourprojectname

Updated property [core/project].


We’re ready to train our model! To kick off training, run the following gcloud command:(This is with TPU)

In [ ]:
!gcloud ml-engine jobs submit training `whoami`_object_detection_`date +%s` \
--job-dir=gs://${YOUR_GCS_BUCKET}/fashion_train \
--packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz,/tmp/pycocotools/pycocotools-2.0.tar.gz \
--module-name object_detection.model_tpu_main \
--runtime-version 1.15 \
--scale-tier BASIC_TPU \
--region=us-central1 \
--python-version=3.7 \
-- \
--model_dir=gs://${YOUR_GCS_BUCKET}/fashion_train \
--pipeline_config_path=gs://${YOUR_GCS_BUCKET}/fashion/pipeline.config

With the command below, we can see our training status and debug if needed

In [ ]:
!gcloud ai-platform jobs describe CurrentJobID

If you have used TPU, you can jump this(This is with GPU)

In [ ]:
!gcloud ml-engine jobs submit training `whoami`_object_detection_eval_validation_`date +%s` \
--job-dir=gs://final_my/fashion_train \
--packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz,/tmp/pycocotools/pycocotools-2.0.tar.gz \
--module-name object_detection.model_main \
--runtime-version 1.15 \
--scale-tier BASIC_GPU \
--region us-central1 \
--python-version=3.7 \
-- \
--model_dir=gs://final_my/fashion_train \
--pipeline_config_path=gs://final_my/fashion/pipeline.config \
--checkpoint_dir=gs://final_my/fashion_train

# Running on mobile with TensorFlow Lite

To make these commands easier to run, let’s set up some environment variables:

In [ ]:
!export CONFIG_FILE=gs://yourbucketname/data/pipeline.config
!export CHECKPOINT_PATH=gs://yourbucketname/train/model.ckpt-2000
!export OUTPUT_DIR=/tmp/tflite

You will need a 1.15 version of tensorflow

In [ ]:
!pip uninstall -y tb-nightly tensorboard tensorflow tensorflow-estimator tf-estimator-nightly tf-nightly
!pip install tensorflow==1.15

This login process maybe needed

In [ ]:
!gcloud auth application-default login

Export model from bucket

In [ ]:
!python ./models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=gs://yourbucketname/fashion/pipeline.config \
--trained_checkpoint_prefix=gs://yourbucketname/fashion_train/model.ckpt-2000 \
--output_directory=$OUTPUT_DIR \
--add_postprocessing_op=true

Install bazel

In [ ]:
!sudo apt install curl gnupg
!curl -fsSL https://bazel.build/bazel-release.pub.gpg | gpg --dearmor > bazel.gpg
!sudo mv bazel.gpg /etc/apt/trusted.gpg.d/
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.10).
gnupg is already the newest version (2.2.4-1ubuntu1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 56 not upgraded.
deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8


In [ ]:
!sudo apt update && sudo apt install bazel-3.1.0

Hit:1 https://storage.googleapis.com/bazel-apt stable InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done
Building dependency tree       
Readin

Configuring TOCO tool

In [ ]:
!git clone https://github.com/tensorflow/tensorflow.git

Cloning into 'tensorflow'...
remote: Enumerating objects: 975845, done.
remote: Total 975845 (delta 0), reused 0 (delta 0), pack-reused 975845
Receiving objects: 100% (975845/975845), 576.78 MiB | 15.06 MiB/s, done.
Resolving deltas: 100% (794757/794757), done.
Checking out files: 100% (23159/23159), done.


In [ ]:
%cd tensorflow

/content/tensorflow


Next we’ll use TensorFlow Lite to get the optimized model by using TOCO, the TensorFlow Lite Optimizing Converter. This will convert the resulting frozen graph (tflite_graph.pb) to the TensorFlow Lite flatbuffer format (detect.tflite) via the following command.

In [ ]:
!bazel run -c opt tensorflow/lite/toco:toco -- \
--input_file=/tmp/tflite/tflite_graph.pb \
--output_file=/tmp/tflite/detect.tflite \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
--inference_type=QUANTIZED_UINT8 \
--mean_values=128 \
--std_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops

Streaming output truncated to the last 5000 lines.
[1,517 / 2,172] 2 actions running
    Compiling tensorflow/core/framework/typed_allocator.cc; 0s local
[1,517 / 2,172] 2 actions running
    Compiling tensorflow/core/framework/typed_allocator.cc; 1s local
[1,517 / 2,172] 2 actions running
    Compiling tensorflow/core/framework/typed_allocator.cc; 2s local
[1,518 / 2,172] 2 actions, 1 running
    Compiling tensorflow/core/framework/log_memory.cc; 3s local
[1,518 / 2,172] 2 actions running
    Compiling tensorflow/core/framework/log_memory.cc; 3s local
[1,519 / 2,172] 2 actions running
    Compiling tensorflow/core/framework/variant_op_registry.cc; 0s local
[1,519 / 2,172] 2 actions running
    Compiling tensorflow/core/framework/variant_op_registry.cc; 0s local
[1,519 / 2,172] 2 actions running
    Compiling tensorflow/core/framework/variant_op_registry.cc; 1s local
[1,519 / 2,172] 2 actions running
    Compiling tensorflow/core/framework/variant_op_registry.cc; 2s local
[1,519 / 2,17

Congratulations！You have gotten the detect.tflite and you can deploy it on Android with Android Studio!

# Reference
https://blog.tensorflow.org/2018/07/training-and-serving-realtime-mobile-object-detector-cloud-tpus.html